# 文本特征及分类

## 1. 情感分析

In [1]:
# 简单的例子

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier

text1 = 'I like the movie so much!'
text2 = 'That is a good movie.'
text3 = 'This is a great one.'
text4 = 'That is a really bad movie.'
text5 = 'This is a terrible movie.'

def proc_text(text):
    """
        预处处理文本
    """
    # 分词
    raw_words = nltk.word_tokenize(text)
    
    # 词形归一化
    wordnet_lematizer = WordNetLemmatizer()    
    words = [wordnet_lematizer.lemmatize(raw_word) for raw_word in raw_words]
    
    # 去除停用词
    filtered_words = [word for word in words if word not in stopwords.words('english')]
    
    # True 表示该词在文本中，为了使用nltk中的分类器
    return {word: True for word in filtered_words}

# 构造训练样本
train_data = [[proc_text(text1), 1],
              [proc_text(text2), 1],
              [proc_text(text3), 1],
              [proc_text(text4), 0],
              [proc_text(text5), 0]]

print(train_data)

[[{'I': True, 'like': True, 'movie': True, 'much': True, '!': True}, 1], [{'That': True, 'good': True, 'movie': True, '.': True}, 1], [{'This': True, 'great': True, 'one': True, '.': True}, 1], [{'That': True, 'really': True, 'bad': True, 'movie': True, '.': True}, 0], [{'This': True, 'terrible': True, 'movie': True, '.': True}, 0]]


In [3]:
# 训练模型
nb_model = NaiveBayesClassifier.train(train_data)

# 测试模型
text6 = 'That is a not bad one.'
print(nb_model.classify(proc_text(text6)))

0


## 2. 文本相似度

In [4]:
import nltk
from nltk import FreqDist

text1 = 'I like the movie so much '
text2 = 'That is a good movie '
text3 = 'This is a great one '
text4 = 'That is a really bad movie '
text5 = 'This is a terrible movie'

text = text1 + text2 + text3 + text4 + text5
words = nltk.word_tokenize(text)
freq_dist = FreqDist(words)
print(freq_dist['That'])

2


In [5]:
freq_dist

FreqDist({'I': 1,
          'That': 2,
          'This': 2,
          'a': 4,
          'bad': 1,
          'good': 1,
          'great': 1,
          'is': 4,
          'like': 1,
          'movie': 4,
          'much': 1,
          'one': 1,
          'really': 1,
          'so': 1,
          'terrible': 1,
          'the': 1})

In [6]:
# 取出常用的n=5个单词
n = 5

# 构造“常用单词列表”
most_common_words = freq_dist.most_common(n)
print(most_common_words)

[('movie', 4), ('is', 4), ('a', 4), ('That', 2), ('This', 2)]


In [7]:
def lookup_pos(most_common_words):
    """
        查找常用单词的位置
    """
    result = {}
    pos = 0
    for word in most_common_words:
        result[word[0]] = pos
        pos += 1
    return result

# 记录位置
std_pos_dict = lookup_pos(most_common_words)
print(std_pos_dict)

{'movie': 0, 'is': 1, 'a': 2, 'That': 3, 'This': 4}


In [8]:
# 新文本
new_text = 'That one is a good movie. This is so good!'

# 初始化向量
freq_vec = [0] * n

# 分词
new_words = nltk.word_tokenize(new_text)

# 在“常用单词列表”上计算词频
for new_word in new_words:
    if new_word in list(std_pos_dict.keys()):
        freq_vec[std_pos_dict[new_word]] += 1

print(freq_vec)

[1, 2, 1, 1, 1]


In [ ]:
new_text = 'That one is a good movie.'

## 3. 文本分类及TF-IDF

### 3.1 NLTK中的TF-IDF

In [9]:
from nltk.text import TextCollection

text1 = 'I like the movie so much '
text2 = 'That is a good movie '
text3 = 'This is a great one '
text4 = 'That is a really bad movie '
text5 = 'This is a terrible movie'

# 构建TextCollection对象
tc = TextCollection([text1, text2, text3, 
                        text4, text5])
new_text = 'That one is a good movie. This is so good!'
word = 'That'
tf_idf_val = tc.tf_idf(word, new_text)
print('{}的TF-IDF值为：{}'.format(word, tf_idf_val))

That的TF-IDF值为：0.02181644599700369


### 3.2 sklearn中的TF-IDF

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
feat = vectorizer.fit_transform([text1, text2, text3, text4, text5])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [12]:
feat.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.48127008,
        0.27113917, 0.48127008, 0.        , 0.        , 0.48127008,
        0.        , 0.        , 0.48127008, 0.        ],
       [0.        , 0.6614376 , 0.        , 0.3726424 , 0.        ,
        0.3726424 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.53364369, 0.        , 0.        ],
       [0.        , 0.        , 0.58042343, 0.32700044, 0.        ,
        0.        , 0.        , 0.58042343, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.46828197],
       [0.55167715, 0.        , 0.        , 0.31080528, 0.        ,
        0.31080528, 0.        , 0.        , 0.55167715, 0.        ,
        0.        , 0.44508965, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.3726424 , 0.        ,
        0.3726424 , 0.        , 0.        , 0.        , 0.        ,
        0.6614376 , 0.        , 0.        , 0.53364369]])

In [13]:
vectorizer.get_feature_names()

['bad',
 'good',
 'great',
 'is',
 'like',
 'movie',
 'much',
 'one',
 'really',
 'so',
 'terrible',
 'that',
 'the',
 'this']

In [ ]:
feat_array = feat.toarray()
feat_array.shape

In [ ]:
feat_array[0:2, :]

In [14]:
vectorizer.transform([new_text]).toarray()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


array([[0.        , 0.67082255, 0.        , 0.37792972, 0.        ,
        0.18896486, 0.        , 0.33541128, 0.        , 0.33541128,
        0.        , 0.27060771, 0.        , 0.27060771]])

### 3.3 中文TF-IDF

In [16]:
import os
import re
import jieba.posseg as pseg

ch_text1 = ' 非常失望，剧本完全敷衍了事，主线剧情没突破大家可以理解，可所有的人物都缺乏动机，正邪之间、妇联内部都没什么火花。团结-分裂-团结的三段式虽然老套但其实也可以利用积攒下来的形象魅力搞出意思，但剧本写得非常肤浅、平面。场面上调度混乱呆板，满屏的铁甲审美疲劳。只有笑点算得上差强人意。'
ch_text2 = ' 2015年度最失望作品。以为面面俱到，实则画蛇添足；以为主题深刻，实则老调重弹；以为推陈出新，实则俗不可耐；以为场面很high，实则high劲不足。气！上一集的趣味全无，这集的笑点明显刻意到心虚。全片没有任何片段给我有紧张激动的时候，太弱了，跟奥创一样。'
ch_text3 = ' 《铁人2》中勾引钢铁侠，《妇联1》中勾引鹰眼，《美队2》中勾引美国队长，在《妇联2》中终于……跟绿巨人表白了，黑寡妇用实际行动告诉了我们什么叫忠贞不二；而且为了治疗不孕不育连作战武器都变成了两支验孕棒(坚决相信快银没有死，后面还得回来)'
ch_text4 = ' 虽然从头打到尾，但是真的很无聊啊。'
ch_text5 = ' 剧情不如第一集好玩了，全靠密集笑点在提神。僧多粥少的直接后果就是每部寡姐都要换着队友谈恋爱，这特么比打斗还辛苦啊，真心求放过～～～（结尾彩蛋还以为是洛基呢，结果我呸！）'

ch_texts = [ch_text1, ch_text2, ch_text3, ch_text4, ch_text5]

In [17]:
# 加载停用词表
stop_words_path = './stop_words/'

stopwords1 = [line.rstrip() for line in open(os.path.join(stop_words_path, '中文停用词库.txt'), 'r',
                                             encoding='utf-8')]
stopwords2 = [line.rstrip() for line in open(os.path.join(stop_words_path, '哈工大停用词表.txt'), 'r',
                                             encoding='utf-8')]
stopwords3 = [line.rstrip() for line in
              open(os.path.join(stop_words_path, '四川大学机器智能实验室停用词库.txt'), 'r', encoding='utf-8')]
stopwords = stopwords1 + stopwords2 + stopwords3

In [18]:
len(stopwords)

2489

In [20]:
def proc_text(raw_line):
    """
        处理文本数据
        返回分词结果
    """

    # 1. 使用正则表达式去除非中文字符
    filter_pattern = re.compile('[^\u4E00-\u9FD5]+')
    chinese_only = filter_pattern.sub('', raw_line)

    # 2. 结巴分词+词性标注
    word_list = pseg.cut(chinese_only)

    # 3. 去除停用词，保留有意义的词性
    # 动词，形容词，副词
    used_flags = ['v', 'a', 'ad']
    meaninful_words = []
    for word, flag in word_list:
        if (word not in stopwords) and (flag in used_flags):
            meaninful_words.append(word)
    return ' '.join(meaninful_words)

In [21]:
corpus = [proc_text(ch_text) for ch_text in ch_texts]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Robin\AppData\Local\Temp\jieba.cache
Loading model cost 0.943 seconds.
Prefix dict has been built succesfully.


In [22]:
corpus

['失望 完全 没 理解 缺乏 正邪 团结 分裂 团结 积攒 搞 出 写 肤浅 混乱 笑 点算',
 '失望 不足 全 明显 刻意 紧张 激动 太弱',
 '勾引 勾引 勾引 表白 告诉 治疗 不孕 不育 作战 变成 验孕 棒 坚决 相信 快 死 回来',
 '打到 无聊',
 '好玩 全 提神 直接 换 谈恋爱 打斗 辛苦 求 放过']

In [23]:
ch_vectorizer = TfidfVectorizer()
ch_feats = ch_vectorizer.fit_transform(corpus)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [25]:
ch_vectorizer.get_feature_names()

['不孕',
 '不育',
 '不足',
 '作战',
 '分裂',
 '刻意',
 '勾引',
 '变成',
 '告诉',
 '回来',
 '团结',
 '坚决',
 '太弱',
 '失望',
 '好玩',
 '完全',
 '打到',
 '打斗',
 '提神',
 '放过',
 '无聊',
 '明显',
 '正邪',
 '治疗',
 '混乱',
 '激动',
 '点算',
 '理解',
 '直接',
 '相信',
 '积攒',
 '紧张',
 '缺乏',
 '肤浅',
 '表白',
 '谈恋爱',
 '辛苦',
 '验孕']

In [26]:
ch_feats.toarray()[0, :]

array([0.        , 0.        , 0.        , 0.        , 0.2706569 ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.5413138 , 0.        , 0.        , 0.21836428, 0.        ,
       0.2706569 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.2706569 , 0.        , 0.2706569 ,
       0.        , 0.2706569 , 0.2706569 , 0.        , 0.        ,
       0.2706569 , 0.        , 0.2706569 , 0.2706569 , 0.        ,
       0.        , 0.        , 0.        ])